In [1]:
####################################################################################################
# wis_dnn_challenge.py
# Description: This is a template file for the WIS DNN challenge submission.
# Important: The only thing you should not change is the signature of the class (Predictor) and its predict function.
#            Anything else is for you to decide how to implement.
#            We provide you with a very basic working version of this class.
#
# Author: <first name1>_<last name1> [<first name1>_<last name2>]
#
# Python 3.7
####################################################################################################

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import sys
from tensorflow.keras.callbacks import ModelCheckpoint


# The time series that you would get are such that the difference between two rows is 15 minutes.
# This is a global number that we used to prepare the data, so you would need it for different purposes.
DATA_RESOLUTION_MIN = 15


class Predictor(object):
    """
    This is where you should implement your predictor.
    The testing script calls the 'predict' function with the glucose and meals test data which you will need in order to
    build your features for prediction.
    You should implement this function as you wish, just do not change the function's signature (name, parameters).
    The other functions are here just as an example for you to have something to start with, you may implement whatever
    you wish however you see fit.
    """

    def __init__(self, path2data):
        """
        This constructor only gets the path to a folder where the training data frames are.
        :param path2data: a folder with your training data.
        """
        self.path2data = path2data
        self.train_glucose = None
        self.train_meals = None
        self.nn = None

    def predict(self, X_glucose, X_meals):
        """
        You must not change the signature of this function!!!
        You are given two data frames: glucose values and meals.
        For every timestamp (t) in X_glucose for which you have at least 12 hours (48 points) of past glucose and two
        hours (8 points) of future glucose, predict the difference in glucose values for the next 8 time stamps
        (t+15, t+30, ..., t+120).

        :param X_glucose: A pandas data frame holding the glucose values in the format you trained on.
        :param X_meals: A pandas data frame holding the meals data in the format you trained on.
        :return: A numpy ndarray, sized (M x 8) holding your predictions for every valid row in X_glucose.
                 M is the number of valid rows in X_glucose (number of time stamps for which you have at least 12 hours
                 of past glucose values and 2 hours of future glucose values.
                 Every row in your final ndarray should correspond to:
                 (glucose[t+15min]-glucose[t], glucose[t+30min]-glucose[t], ..., glucose[t+120min]-glucose[t])
        """

        # build features for set of (ID, timestamp)
        ids = self.cgm_and_meals_id_time.reset_index()['id'].unique()        
        x_all, y_all, id_all,x_GV= self.build_features(ids=ids)

        #load nn:
        model=load_nn_model(self)        
       
        # feed the network you trained
        y_predict = model.predict([x_GV,x_all])
       
        #create data frame with idx:
        y=pd.DataFrame(y_predict,index=id_all.ravel().astype(int))
        y.index.names = ['id']
       
        return y
    
    
    def define_nn_hypPars(self, BATCH_SIZE = 32, EVALUATION_INTERVAL = 10000, EPOCHS = 10, num_filters = [6,12,24,36,48,60,72,84,96],
                          kernel_size = [3,3,3,3,3,3,3,3,3], LR = 0.1, n_LSTM = [48,16,16,16,16,16], n_dense = [16,16,16,16,16]):
        """
        Define your neural network.
        :return: None
        """
       
        self.BATCH_SIZE = BATCH_SIZE
        self.EVALUATION_INTERVAL = EVALUATION_INTERVAL
        self.EPOCHS = EPOCHS
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.LR = LR
        self.n_LSTM = n_LSTM
        self.n_dense = n_dense

        pass
    
    

    def define_nn(self,GV_shape,MealsAndGV_shape):
        
        """
        Define your neural network.
        :return: None
        """
        
        ## Get Hyper parameters
        #Predictor.define_nn_hypPars(self)
        
        #### A split model - LSTM for GV, CNN for GV+Meals, FC after concatenation using functional API

        ## define inputs for the model
        GV_input = tf.keras.layers.Input(shape=GV_shape, name='GVinput')
        MealsAndGV_input = tf.keras.layers.Input(shape=MealsAndGV_shape, name='MealsAndGVinput')

        ## Build first branch - LSTM + FC for GV data
        activation_GV = 'relu'

        GV_lstm1 = tf.keras.layers.LSTM(self.n_LSTM[0],return_sequences=True)(GV_input)
        GV_lstm2 = tf.keras.layers.LSTM(self.n_LSTM[1])(GV_lstm1)
        GVoutput = tf.keras.layers.Dense(self.n_dense[0],activation=activation_GV,name='GVoutput')(GV_lstm2)
        # GVoutput = tf.keras.layers.BatchNormalization()(GV_Dense)

        ## Build second branch - CNN + FC for Meals and GV data
        activation_MealsAndGV = 'relu'
        MealsAndGV_conv1 = tf.keras.layers.Conv1D(self.num_filters[0],self.kernel_size[0], 
                                                  activation=activation_MealsAndGV)(MealsAndGV_input)
        MealsAndGV_conv2 = tf.keras.layers.Conv1D(self.num_filters[1],self.kernel_size[1], 
                                                  activation=activation_MealsAndGV)(MealsAndGV_conv1)
        MealsAndGV_conv3 = tf.keras.layers.Conv1D(self.num_filters[2],self.kernel_size[2], 
                                                  activation=activation_MealsAndGV)(MealsAndGV_conv2)
        MealsAndGV_conv4 = tf.keras.layers.Conv1D(self.num_filters[3],self.kernel_size[3], 
                                                  activation=activation_MealsAndGV)(MealsAndGV_conv3)
        # MealsAndGV_pool = tf.keras.layers.MaxPooling1D(2)(MealsAndGV_conv_third)
        MealsAndGV_flat = tf.keras.layers.Flatten()(MealsAndGV_conv4)
        MealsAndGV_output = tf.keras.layers.Dense(self.n_dense[1],activation=activation_MealsAndGV,name='Mealsoutput')(MealsAndGV_flat)
        # MealsAndGV_output = tf.keras.layers.BatchNormalization()(MealsAndGV_conv_second)

        ## Concatenate branches and generate output
        activation_concat = 'relu'
        combined = tf.keras.layers.concatenate([GVoutput, MealsAndGV_output])

        combined_Dense1 = tf.keras.layers.Dense(self.n_dense[2],activation=activation_concat)(combined)
        combined_output = tf.keras.layers.Dense(8,name='combined_output')(combined_Dense1)
        
        ## Build and compile the Final model
        self.nn = tf.keras.Model(inputs=[GV_input,MealsAndGV_input], outputs=[combined_output]) 
        self.nn.compile(optimize='adam',loss='mae',learning_rate = self.LR)

        return


    def train_nn(self, X_train, y_train):

        """
        Train your network using the training data.
        :param X_train: A pandas data frame holding the features
        :param y_train: A numpy ndarray, sized (M x 8) holding the values you need to predict.
        :return:
        """


        ## Get Hyper parameters
        #Predictor.define_nn_hypPars(self)

        ## Split the input
        X_train_GV = X_train[:,:,0:1]

        ## Define callbacks to save data during training
        checkpointer = ModelCheckpoint(filepath=self.path2data+'{epoch:02d}-'+str(self.BATCH_SIZE)+
                           '-'+str(self.n_LSTM[0])+'.hdf5',
                           save_best_only=False)

        ## Train the network
        self.nn.fit([X_train_GV,X_train],y_train,epochs=self.EPOCHS,
                    batch_size=self.BATCH_SIZE,steps_per_epoch=self.EVALUATION_INTERVAL,
                   callbacks=[checkpointer])

        # save_nn_model(self)

        pass
    

    def save_nn_model(self):
        """
        Save your neural network after training.
        :return:
        """
        pass

    def load_nn_model_for_train(self,epoch_str):
        """
        Load your trained neural network.
        :return:
        """
        self.current_model = tf.keras.models.load_model(self.path2data+epoch_str+'-'+str(self.BATCH_SIZE)+
                               '-'+str(self.n_LSTM[0])+'.hdf5')

        pass
    
    def load_nn_model(self):
        """
        Load your trained neural network.
        :return:
        """

        pass

    @staticmethod
    def load_data_frame(path):
        """
        Load a pandas data frame in the relevant format.
        :param path: path to csv.
        :return: the loaded data frame.
        """
        return pd.read_csv(path, index_col=[0, 1], parse_dates=['Date'])

    def load_raw_data(self,train=False):
        """
        Loads raw data frames from csv files, and do some basic cleaning
        :return:
        """
        self.train_glucose = Predictor.load_data_frame(os.path.join(self.path2data, 'GlucoseValues.csv'))
        self.train_meals = Predictor.load_data_frame(os.path.join(self.path2data, 'Meals.csv'))

        # remove food_id and unit_id columns from Meals df
        self.train_meals = self.train_meals.drop(['meal_type','unit_id','food_id'],axis=1)
       
       
        if train:
        # define  normaliation function, assumes an 'id' feature in dataframe!
            def normalize_df (df):
                mean_pd_by_id = df.groupby('id').mean()
                std_pd_by_id = df.groupby('id').std()

                df['Norm_GV'] = (df-mean_pd_by_id)/std_pd_by_id

                return df

            # normalize GV for every individual
            self.train_glucose = normalize_df (self.train_glucose)

            # remove outliers - thr std's above the mean
            thr_std_outlier = 4
            self.train_glucose = self.train_glucose[self.train_glucose['Norm_GV'].abs() < thr_std_outlier]

            # re-normalize after removing outliers
            self.train_glucose = normalize_df (self.train_glucose)
           
            #remove normalization:
            self.train_glucose= self.train_glucose.drop(['Norm_GV'],axis=1)
                 
            # remove outliers of meals - thr std's above the mean
            thr_std = 4
            thr_outlier = self.train_meals.copy().replace(0,np.nan)
            thr_outlier = thr_outlier.groupby('id').mean() + thr_std * thr_outlier.groupby('id').std()
            self.train_meals = self.train_meals[self.train_meals - thr_outlier < 0]
           
                  

        # resample meals using the timestamps from the glucose df
        timeStamp = str(DATA_RESOLUTION_MIN)+'T'
        self.train_meals = self.train_meals.groupby(pd.Grouper(level=0)).resample(timeStamp,level=-1).sum()

        # Normalize after removing outliers
        self.train_meals = self.train_meals/self.train_meals.groupby('id').max()

        # resample glucose dataframe in timestamps identical to meals
        self.train_glucose = self.train_glucose.groupby(pd.Grouper(level=0)).resample(timeStamp,level=-1).first()        
       
        # replace NAN values in Glucose with linear interpolation
        self.train_glucose = self.train_glucose.interpolate()
                
        # now merge the glucose and meals dataframes on the same time stamps
        self.cgm_and_meals_id_time = self.train_glucose.merge(self.train_meals,how='left',left_index=True,right_index=True)

        # replace NAN values in meals with 0's
        self.cgm_and_meals_id_time = self.cgm_and_meals_id_time.fillna(0)
        self.X_glucose=self.cgm_and_meals_id_time.iloc[:,0:1]
        self.X_meals=self.cgm_and_meals_id_time.iloc[:,1:]
     
        return self.X_glucose, self.X_meals
   
   
    def build_features(self,ids, build_y=False, n_previous_time_points=48):
        """
        Given glucose and meals data, build the features needed for prediction.
        :param X_glucose: A pandas data frame holding the glucose values.
        :param X_meals: A pandas data frame holding the meals data.
        :param build_y: Whether to also extract the values needed for prediction.
        :param n_previous_time_points:
        :return: The features needed for your prediction, and optionally also the relevant y arrays for training.
        """
        # function that create shifts:
        def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
            data = []
            labels = []
            start_index=0
            start_index = start_index + history_size
            if end_index is None:
                end_index = len(dataset) - target_size
     

            for i in range(start_index, end_index):
                indices = range(i-history_size, i, step)
                data.append(dataset[indices])

                if single_step:
                   labels.append(target[i+target_size])
                else:
                   labels.append(target[i:i+target_size]-target[i-1])

            return np.array(data), np.array(labels)

        # ,,,,,,,,,
        past_history = n_previous_time_points
        future_target = 8
        STEP = 1

   
        x_all = np.empty([1,past_history,self.cgm_and_meals_id_time.shape[1]])
        y_all = np.empty([1,future_target])
        id_all = np.empty([1,1])

        for id in ids:
            multi_data = self.cgm_and_meals_id_time.loc[id]
            end_index = len(multi_data)-future_target
            x_single, y_single = multivariate_data(multi_data.values, multi_data.GlucoseValue.values, 0,
                                                   len(multi_data.values)-future_target, past_history,
                                                   future_target, STEP)
                                                   
            x_all = np.append(x_all,x_single,axis=0)
            y_all = np.append(y_all,y_single,axis=0)
            id_all = np.append(id_all,id*np.ones([y_single.shape[0],1]),axis=0)
   
        # remove first empty example
        x_all = x_all[1:]
        y_all = y_all[1:]
        id_all = id_all[1:]
        x_GV=x_all[:,:,0:1]
       
        return x_all, y_all, id_all,x_GV


In [2]:
if __name__ == "__main__":
    # example of predict() usage

    # create Predictor instance
    path2data = ''
    predictor = Predictor(path2data)

    # load the raw data
    predictor.load_raw_data(train=True)



In [3]:
ids_all = predictor.cgm_and_meals_id_time.reset_index()['id'].unique()
# split the data to train and cv - use 80% as train
TRAIN_SPLIT = 0.8
ids_train = ids_all[range(round(len(ids_all)*TRAIN_SPLIT))]
ids_cv = ids_all[:-(round(len(ids_all)*TRAIN_SPLIT))]
data_train=predictor.cgm_and_meals_id_time

x_train_all, y_train_all, id_train_all,x_train_GV=predictor.build_features(ids=ids_train)
x_cv_all, y_cv_all, id_cv_all,x_cv_GV=predictor.build_features(ids=ids_cv)


In [4]:
# save data to file
np.savez_compressed('train_all_data_no_normalization', x_train_all=x_train_all, y_train_all=y_train_all,
                    id_train_all=id_train_all,x_train_GV=x_train_GV)
np.savez_compressed('CV_all_data_no_normalization', x_cv_all=x_cv_all, y_cv_all=y_cv_all,
                    id_cv_all=id_cv_all,x_cv_GV=x_cv_GV)


In [3]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)
    data.append(dataset[indices])

    if single_step:
      labels.append(target[i+target_size])
    else:
      labels.append(target[i:i+target_size]-target[i-1])

  return np.array(data), np.array(labels)

In [19]:
past_history = 12*4
future_target = 8
STEP = 1

# list of id's
ids_all = predictor.cgm_and_meals_id_time.reset_index()['id'].unique()

# split the data to train and cv - use 80% as train
TRAIN_SPLIT = 0.8
ids_train = ids_all[range(round(len(ids_all)*TRAIN_SPLIT))]
ids_cv = ids_all[:-(round(len(ids_all)*TRAIN_SPLIT))]

# ids_train = ids_all[-round(len(ids_all)*0.8):]

x_train_all = np.empty([1,past_history,predictor.cgm_and_meals_id_time.shape[1]])
y_train_all = np.empty([1,future_target])
id_train_all = np.empty([1,1])

for id in ids_train:
    multi_data = predictor.cgm_and_meals_id_time.loc[id]
    end_index = len(multi_data)-future_target
    x_train_single, y_train_single = multivariate_data(multi_data.values, multi_data.GlucoseValue.values, 0,
                                                   len(multi_data.values)-future_target, past_history,
                                                   future_target, STEP,
                                                   single_step=False)
    x_train_all = np.append(x_train_all,x_train_single,axis=0)
    y_train_all = np.append(y_train_all,y_train_single,axis=0)
    id_train_all = np.append(id_train_all,id*np.ones([y_train_single.shape[0],1]),axis=0)
    
# remove first empty example
x_train_all = x_train_all[1:]
y_train_all = y_train_all[1:]
id_train_all = id_train_all[1:]

In [4]:
def concatenateIndividuals (self,ids,STEP=1,past_history=48,future_target=8):
    
    x_all = np.empty([1,past_history,predictor.cgm_and_meals_id_time.shape[1]])
    y_all = np.empty([1,future_target])
    id_all = np.empty([1,1])

    for id in ids_train:
        multi_data = predictor.cgm_and_meals_id_time.loc[id]
        end_index = len(multi_data)-future_target
        x_single, y_single = multivariate_data(multi_data.values, multi_data.GlucoseValue.values, 0,
                                                       len(multi_data.values)-future_target, past_history,
                                                       future_target, STEP,
                                                       single_step=False)
        x_train_all = np.append(x_train_all,x_train_single,axis=0)
        y_train_all = np.append(y_train_all,y_train_single,axis=0)
        id_train_all = np.append(id_train_all,id*np.ones([y_train_single.shape[0],1]),axis=0)

    # remove first empty example
    x_all = x_all[1:]
    y_all = y_all[1:]
    id_all = id_all[1:]
    
    return x_all, y_all, id_all

In [20]:
# Prepare the CV data
x_CV_all = np.empty([1,past_history,predictor.cgm_and_meals_id_time.shape[1]])
y_CV_all = np.empty([1,future_target])
id_CV_all = np.empty([1,1])

for id in ids_cv:
    multi_cv_data = predictor.cgm_and_meals_id_time.loc[id]
    end_index = len(multi_cv_data)-future_target
    x_CV_single, y_CV_single = multivariate_data(multi_cv_data.values, multi_cv_data.GlucoseValue.values, 0,
                                                   len(multi_cv_data.values)-future_target, past_history,
                                                   future_target, STEP,
                                                   single_step=False)
    x_CV_all = np.append(x_CV_all,x_CV_single,axis=0)
    y_CV_all = np.append(y_CV_all,y_CV_single,axis=0)
    id_CV_all = np.append(id_CV_all,id*np.ones([y_CV_single.shape[0],1]),axis=0)

# remove first empty example
x_CV_all = x_CV_all[1:]
y_CV_all = y_CV_all[1:]
id_CV_all = id_CV_all[1:]

In [5]:
# load data saved as .npz files
#d1 = np.load('train_all_data.npz')
#x_train_all_norm = d1['x_train_all']
#y_train_all_norm = d1['y_train_all']
#id_train_all = d1['id_train_all']

#d2 = np.load('CV_all_data.npz')
#x_cv_all_norm = d2['x_CV_all']
#y_cv_all_norm = d2['y_CV_all']
#id_cv_all = d2['id_CV_all']

d3 = np.load('train_all_data_no_normalization.npz')
x_train_all = d3['x_train_all']
y_train_all = d3['y_train_all']
id_train_all = d3['id_train_all']
x_train_GV = d3['x_train_GV']

d4 = np.load('CV_all_data_no_normalization.npz')
x_cv_all = d4['x_cv_all']
y_cv_all = d4['y_cv_all']
id_cv_all = d4['id_cv_all']
x_cv_GV = d4['x_cv_GV']


In [5]:
# x_train_GV = x_train_all[:,:,0:2]
# x_train_Meals = x_train_all[:,:,2:]

# x_cv_GV = x_cv_all[:,:,0:2]
# x_cv_Meals = x_cv_all[:,:,2:]

In [6]:
# evaluate prediction accuracy using correlation
def estimatePredictionUsingCorrelation (x,y,model,y_cv_baseline):

    y_predict = model.predict(x)

    corr_predition=np.corrcoef(y_predict.flatten(),y.flatten())

    import numpy.matlib

    y_cv_baseline=np.matlib.repmat(y_cv_baseline,8,1)
    y_cv_baseline=np.transpose(y_cv_baseline)

    y_cv_mean=np.mean(x[:,:,0],axis=1)
    y_cv_mean=np.matlib.repmat(y_cv_mean,8,1)
    y_cv_mean=np.transpose(y_cv_mean)

    corr_baseline=np.corrcoef(y_cv_baseline.flatten(),y.flatten())
    corr_mean=np.corrcoef(y_cv_mean.flatten(),y.flatten())
    corr_mean = 0
    return corr_predition,corr_baseline, corr_mean

In [6]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_all, y_train_all))
train_data_multi = train_data_multi.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_multi = tf.data.Dataset.from_tensor_slices((x_cv_all, y_cv_all))
val_data_multi = val_data_multi.batch(BATCH_SIZE).repeat()

In [7]:
# try a model with convolutional layers
multi_step_model_conv = tf.keras.models.Sequential()
multi_step_model_conv.add(tf.keras.layers.Conv1D(3,8, input_shape=x_train_all.shape[-2:])) #,
                                         # stateful = 'stateful',
                                         # batch_size=BATCH_SIZE))
multi_step_model_conv.add(tf.keras.layers.Activation('relu'))
multi_step_model_conv.add(tf.keras.layers.Conv1D(6,4))
multi_step_model_conv.add(tf.keras.layers.Activation('relu'))
multi_step_model_conv.add(tf.keras.layers.MaxPooling1D(pool_size=2))
multi_step_model_conv.add(tf.keras.layers.Flatten())
# multi_step_model_conv.add(tf.keras.layers.Activation('relu'))
# multi_step_model_conv.add(tf.keras.layers.AveragePooling1D(4))
# multi_step_model_conv.add(tf.keras.layers.Conv1D(16,2)) #,
# multi_step_model_conv.add(tf.keras.layers.Activation('relu'))
# multi_step_model_conv.add(tf.keras.layers.AveragePooling1D(2))
multi_step_model_conv.add(tf.keras.layers.Dense(16,activation='relu'))
multi_step_model_conv.add(tf.keras.layers.Dense(8)) #,activation='softmax'))

In [8]:
multi_step_model_conv.compile(optimizer='adam', loss='mae')

In [9]:
EPOCHS = 5
EVALUATION_INTERVAL = 200
multi_step_history_conv = multi_step_model_conv.fit(train_data_multi, epochs=EPOCHS,
                                          steps_per_epoch=EVALUATION_INTERVAL,
                                          validation_data=val_data_multi,
                                          validation_steps=50)

Train for 200 steps, validate for 50 steps
Epoch 1/5
200/200 [==============================] - 11s 56ms/step - loss: 12.3126 - val_loss: 15.9269
Epoch 2/5
200/200 [==============================] - 3s 13ms/step - loss: 11.3649 - val_loss: 15.3399
Epoch 3/5
200/200 [==============================] - 3s 13ms/step - loss: 11.2900 - val_loss: 14.7320
Epoch 4/5
200/200 [==============================] - 3s 13ms/step - loss: 9.9901 - val_loss: 14.2285
Epoch 5/5
200/200 [==============================] - 3s 14ms/step - loss: 10.2159 - val_loss: 14.4405


In [14]:
corr_predition,corr_baseline,corr_mean = estimatePredictionUsingCorrelation (x_cv_all,y_cv_all,multi_step_model_conv,x_cv_all[:,:,1].mean(axis=1)-x_cv_all[:,47,1])

In [15]:
print('Prediction accuracy (correlation): ' + str(corr_predition[0,1]))
print('Baseline correlation (last GV measure): ' + str(corr_baseline[0,1]))

Prediction accuracy (correlation): 0.4690251465756874
Baseline correlation (last GV measure): 0.4510631920946048


In [54]:
# try the LSTM model using tf DataSet
#create the model
multi_step_model_LSTM = tf.keras.models.Sequential()
multi_step_model_LSTM.add(tf.keras.layers.LSTM(48,
                                          return_sequences=True,
                                          input_shape=x_train_all.shape[-2:])) #,
                                         # stateful = 'stateful',
                                         # batch_size=BATCH_SIZE))
multi_step_model_LSTM.add(tf.keras.layers.LSTM(16, activation='relu'))
# multi_step_model.add(tf.keras.layers.Dense(16))
multi_step_model_LSTM.add(tf.keras.layers.Dense(8))

#multi_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(clipvalue=1.0), loss='mae')
multi_step_model_LSTM.compile(optimizer='adam', loss='mae')

# run model
multi_step_history_LSTM = multi_step_model_LSTM.fit(train_data_multi, epochs=EPOCHS,
                                          steps_per_epoch=EVALUATION_INTERVAL,
                                          validation_data=val_data_multi,
                                          validation_steps=50)

Train for 200 steps, validate for 50 steps
Epoch 1/5
200/200 [==============================] - 21s 106ms/step - loss: 12.3093 - val_loss: 15.8837
Epoch 2/5
200/200 [==============================] - 12s 62ms/step - loss: 10.1993 - val_loss: 14.0386
Epoch 3/5
200/200 [==============================] - 13s 67ms/step - loss: 10.0877 - val_loss: 14.0771
Epoch 4/5
200/200 [==============================] - 13s 65ms/step - loss: 9.0913 - val_loss: 13.7967
Epoch 5/5
200/200 [==============================] - 15s 74ms/step - loss: 9.3318 - val_loss: 13.9794


In [63]:
corr_predition,corr_baseline,corr_mean = estimatePredictionUsingCorrelation (x_cv_all,y_cv_all,multi_step_model_LSTM,x_cv_all[:,:,1].mean(axis=1)-x_cv_all[:,47,1])

In [65]:
print('Prediction accuracy (correlation): ' + str(corr_predition[0,1]))
print('Baseline correlation (last GV measure): ' + str(corr_baseline[0,1]))


Prediction accuracy (correlation): 0.5511296488283757
Baseline correlation (last GV measure): 0.4510631920946048


In [26]:
# try the LSTM model using functional API
#create the model
input_API = tf.keras.layers.Input(shape=x_train_all.shape[-2:], name='input')

# API_cond_first = tf.keras.layers.Conv1D(3,8, activation='relu')(input_API)
API_lstm_first = tf.keras.layers.LSTM(48,return_sequences=True)(input_API)
API_lstm_second = tf.keras.layers.LSTM(16,return_sequences=True, activation='relu')(API_lstm_first)
# API_BN = tf.keras.layers.BatchNormalization()(API_lstm_second)
Meals_conv_first = tf.keras.layers.Conv1D(3,8, activation='relu')(API_lstm_second)
# Meals_conv_second = tf.keras.layers.Conv1D(6,4, activation='relu')(Meals_conv_first)
# Meals_conv_third = tf.keras.layers.Conv1D(8,2, activation='relu')(Meals_conv_second)
# Meals_pool = tf.keras.layers.MaxPooling1D(2)(Meals_conv_third)
Meals_flat = tf.keras.layers.Flatten()(Meals_conv_first)
API_Dense = tf.keras.layers.Dense(8,name='output')(Meals_flat)


EPOCHS = 5
EVALUATION_INTERVAL = 200

API_LSTM_model = tf.keras.Model(inputs=[input_API], outputs=[API_Dense]) 
API_LSTM_model.compile(optimize='adam',loss='mae')

API_LSTM = API_LSTM_model.fit([x_train_all],y_train_all,epochs=EPOCHS,
                              batch_size=BATCH_SIZE,steps_per_epoch=EVALUATION_INTERVAL,
                              validation_data = [x_cv_all,y_cv_all],
                              validation_steps = 50)



Train on 660808 samples, validate on 164462 samples
Epoch 1/5
 12800/660808 [..............................] - ETA: 9:53 - loss: 10.6862 - val_loss: 0.0000e+00

In [27]:
corr_predition,corr_baseline,corr_mean = estimatePredictionUsingCorrelation (x_cv_all,y_cv_all,API_LSTM_model,x_cv_all[:,:,1].mean(axis=1)-x_cv_all[:,47,1])

In [28]:
print('Prediction accuracy (correlation): ' + str(corr_predition[0,1]))
print('Baseline correlation (last GV measure): ' + str(corr_baseline[0,1]))

Prediction accuracy (correlation): 0.00017564784582054625
Baseline correlation (last GV measure): 0.4510631920946048


In [6]:
# try a split model - LSTM for GV, CNN for Meals, FC after concatenation using functional API
BATCH_SIZE = 64
#create the model
# Split net - first branch LSTM + FC for GV, second branch FC for Meals
GV_input = tf.keras.layers.Input(shape=x_train_GV.shape[-2:], name='GVinput')
Meals_input = tf.keras.layers.Input(shape=x_train_all.shape[-2:], name='Mealsinput')

activation = 'relu'

# GV branch
GV_lstm_first = tf.keras.layers.LSTM(48,return_sequences=True)(GV_input)
GV_lstm_second = tf.keras.layers.LSTM(16)(GV_lstm_first)
GVoutput = tf.keras.layers.Dense(16,activation='relu',name='GVoutput')(GV_lstm_second)
# GVoutput = tf.keras.layers.BatchNormalization()(GV_Dense)

# Meals branch
Meals_conv1 = tf.keras.layers.Conv1D(6,5, activation='relu')(Meals_input)
Meals_conv2 = tf.keras.layers.Conv1D(12,3, activation='relu')(Meals_conv1)
Meals_conv3 = tf.keras.layers.Conv1D(24,3, activation='relu')(Meals_conv2)
Meals_conv4 = tf.keras.layers.Conv1D(36,3, activation='relu')(Meals_conv3)
# Meals_pool = tf.keras.layers.MaxPooling1D(2)(Meals_conv_third)
Meals_flat = tf.keras.layers.Flatten()(Meals_conv4)
Meals_output = tf.keras.layers.Dense(16,activation=activation,name='Mealsoutput')(Meals_flat)
# Meals_output = tf.keras.layers.BatchNormalization()(Meals_conv_second)

combined = tf.keras.layers.concatenate([GVoutput, Meals_output])

combined_Dense1 = tf.keras.layers.Dense(16,activation=activation)(combined)
combined_Dense2 = tf.keras.layers.Dense(16,activation=activation)(combined_Dense1)
combined_output = tf.keras.layers.Dense(8,name='combined_output')(combined_Dense2)


EPOCHS = 10
EVALUATION_INTERVAL = 2000

combine_model = tf.keras.Model(inputs=[GV_input,Meals_input], outputs=[combined_output]) 
combine_model.compile(optimize='adam',loss='mae')

combine_API_model = combine_model.fit([x_train_GV,x_train_all],y_train_all,epochs=EPOCHS,
                              batch_size=BATCH_SIZE,steps_per_epoch=EVALUATION_INTERVAL)#,
                              #validation_data = ([x_CV_GV,x_CV_Meals],y_CV_all),
                             # validation_steps = 50)



Train on 660808 samples
Epoch 1/10
128000/660808 [====>.........................] - ETA: 8:00 - loss: 8.8170

In [8]:
y_predict = combine_model.predict([x_cv_GV,x_cv_all])

corr_predition=np.corrcoef(y_predict.flatten(),y_cv_all.flatten())

In [9]:
print('Prediction accuracy (correlation): ' + str(corr_predition[0,1]))

Prediction accuracy (correlation): 0.5861511795894495


In [ ]:
max_batch_size = 500
max_num_filters  = [x * 3 for x in [6, 12, 24, 36, 48, 60, 72, 84, 96] ] # do we want increasing size?
max_kernel_size = [x * 4 for x in [6, 6, 6, 6, 6, 6, 6]]
max_LR = 0.3
max_n_LSTM = [x * 4 for x in [48, 16, 16, 16, 16, 16]]
max_n_dense = [x * 4 for x in [16, 16, 16, 16, 16]]

EPOCHS = 10

batch_size = np.empty(1000)
EVALUATION_INTERVAL = np.empty(1000)
num_filters = np.empty([1000,len(max_num_filters)])
kernel_size = np.empty([1000,len(max_kernel_size)])
LR = np.empty(1000)
n_LSTM = np.empty([1000,len(max_n_LSTM)])
n_dense = np.empty([1000,len(max_n_dense)])


for ii_ite in list(range(1,1000)):
   
    tmp_batch_size = int(round((max_batch_size*np.random.rand())))
    batch_size[ii_ite] = tmp_batch_size
    
    #EVALUATION_INTERVAL = (max_EVALUATION_INTERVAL*rand_nums[1]).round()
    tmp_EVALUATION_INTERVAL=(660000/(batch_size[ii_ite]*5)).round().astype(int)
    EVALUATION_INTERVAL[ii_ite] = tmp_EVALUATION_INTERVAL
    
    EPOCHS = (EPOCHS*batch_size[ii_ite]/100).round().astype(int)
    if EPOCHS<10:
        EPOCHS=10
        
    tmp_num_filters = (max_num_filters*np.random.rand(len(max_num_filters))).round().astype(int)
    num_filters[ii_ite,:] = tmp_num_filters
    
    tmp_kernel_size = (max_kernel_size*np.random.rand(len(max_kernel_size))).round().astype(int).tolist()
    kernel_size[ii_ite,:] = tmp_kernel_size
    
    LR[ii_ite] = max_LR*np.random.rand(1)
    
    tmp_n_LSTM = (max_n_LSTM*np.random.rand(len(max_n_LSTM))).round().astype(int)
    n_LSTM[ii_ite,:] = tmp_n_LSTM

    tmp_n_dense = (max_n_dense*np.random.rand(len(max_n_dense))).round().astype(int)
    n_dense[ii_ite,:] = tmp_n_dense
   
   
    #train (make sure to save loss):
    predictor.define_nn_hypPars(BATCH_SIZE = tmp_batch_size, EVALUATION_INTERVAL = tmp_EVALUATION_INTERVAL, 
                                EPOCHS = EPOCHS, num_filters = tmp_num_filters,kernel_size = tmp_kernel_size, 
                                LR = LR[ii_ite], n_LSTM = tmp_n_LSTM, n_dense = tmp_n_dense)
    predictor.define_nn(GV_shape=x_train_GV.shape[-2:],MealsAndGV_shape=x_train_all.shape[-2:])
    
    predictor.train_nn(X_train=x_train_all,y_train=y_train_all)
    del predictor.nn

        


Train on 660808 samples
Epoch 1/10
 21870/660808 [..............................] - ETA: 8:16 - loss: 10.7275

In [11]:
        
predictor.nn = tf.keras.models.load_model(predictor.path2data+'123-337-49.hdf5')

y_predict = predictor.nn.predict([x_cv_GV,x_cv_all])

corr_predition=np.corrcoef(y_predict.flatten(),y_cv_all.flatten())
print('Prediction accuracy (correlation): ' + str(corr_predition[0,1]))

Prediction accuracy (correlation): 0.6503797284441221


AttributeError: 'Predictor' object has no attribute 'nn'